# Lab 1 - drzewa decyzyjne
*"pro" version*

Autorstwa:
- Wojciech Kot  151879
- Julia Samp    151775
z grupy L7, zajęcia piątkowe 11:45

Step - by - step - Wersja podstawowa (pro znajduje się poniżej)

Wpierw, wgranie danych do dataframe'u zapewnianego przez pandas
(ze względu na wygodę, jaką zapewnia oraz obsługę .csv)

In [22]:
import pandas


def prepare_pandas_df_for_basic_problem(path):
    data = pandas.read_csv(path)
    data = data.drop(['PassengerId', 'Name'], axis=1)
    data['Age'] = pandas.cut(data['Age'], bins=[0, 20, 40, 100], labels=['young', 'middle', 'old'], right=True)
    return data


dane2 = prepare_pandas_df_for_basic_problem("titanic-homework.csv")
print(dane2)

    Pclass     Sex     Age  SibSp  Parch  Survived
0        3    male  middle      1      0         0
1        1  female  middle      1      0         1
2        3  female  middle      0      0         1
3        1  female  middle      1      0         1
4        3    male  middle      0      0         0
..     ...     ...     ...    ...    ...       ...
95       3    male     old      0      0         0
96       1    male     old      0      0         0
97       1    male  middle      0      1         1
98       2  female  middle      0      1         1
99       2    male  middle      1      0         0

[100 rows x 6 columns]


Kiedy mamy już poprawnie załadowane dane, należy zacząć budować drzewo, jednak najpierw kilka funkcji, które mogą nam się przydać przy obsłudze dataframe'u:

In [23]:
# Aby wiedzieć ILE i jakie rozgałęzienia będzie miało nasze drzewo, gdy podzielimy na tym atrybucie
def possible_values(df, column_name):
    # zwraca posortowaną listę unikalnych wartości z danej kolumny
    return sorted(df[column_name].unique().tolist())


# Filtr odsiewający kolumnę Survived
def get_column_names(df):
    # zwraca listę kolumn, z wyłączeniem kolumny "z decyzją"
    return [col for col in df.columns if col != "Survived"]


# Funkcja zliczająca ile osób przeżyło i ile nie przeżyło, dla danej wartości w danej kolumnie, używana w split()
def filter_count(df, column_name, value):
    filtered_df = df[df[column_name] == value]
    p = int(filtered_df['Survived'].sum())
    ile_wszystkich = len(filtered_df)
    # zwraca parę [przeżyło, nie-przeżyło]
    return [p, ile_wszystkich - p]


# Funkcja przygotowująca dane w przypadku podziału na danym atrybucie (danej kolumnie)
def split(df, column_name):
    l = []
    for val in possible_values(df, column_name):
        l.append(filter_count(df, column_name, val))
    # zwraca listę par [przeżyli, nie-przeżyli], dla każdego atrybutu
    return l


A teraz funkcje matematyczne z zajęć, które będą nam potrzebne przy podejmowaniu decyzji podczas budowania drzewa

In [24]:
import math

def entropy(lista):
    e = 0
    suma = sum(lista)
    for i in lista:
        if i != 0:
            e -= (i / suma) * math.log(i / suma, 2)
    return e


def conditional_entropy(dane):
    ce = 0
    ile = 0
    for i in dane:
        ile += sum(i)
    for i in dane:
        e = entropy(i)
        ce += e * sum(i) / ile
    return ce


def information_gain(dane):
    y = 0
    n = 0
    for i in dane:
        y += i[0]
        n += i[1]
    ig = entropy([y, n]) - conditional_entropy(dane)

    return ig


# Zawiera w sobie IntrinsicInfo??? (chyba) #todo Smoku to Twoja funkcja, opisz proszę swoje rzeczy
def gain_ratio(dane):
    ig = information_gain(dane)
    ii = 0
    ile = 0
    for i in dane:
        ile += sum(i)
    for i in dane:
        ii -= (sum(i) / ile) * math.log(sum(i) / ile, 2)
    gr = ig/ii
    return gr

Oraz wreszcie główna funkcja służąca do budowy drzewa decyzyjnego:

In [25]:
def build_tree(df, tree, columns, parent=None, parent_id="root"):
    best_gain = 0.0
    best_column = None

    for column in columns:
        ig = information_gain(split(df, column))
        if ig > best_gain:
            best_gain = ig
            best_column = column

    # warunek stopu rekurencji
    if best_gain == 0 or len(columns) == 0:
        return

    # odfiltrowanie atrybutu, na podstawie którego właśnie dokonaliśmy podziału
    remaining_columns = columns.copy()
    remaining_columns.remove(best_column)

    # unikalne node_id składa się z nazwy kolumny, na jakiej robimy podział w tym rozgałęzieniu, oraz id rodzica w drzewie, ponieważ bez tej drugiej części id nie byłoby unikalne
    node_id = f"{best_column}_{parent_id}"
    tree.create_node(best_column, node_id, parent=parent)

    # A tu rekurencyjne wywołanie, aby zbudować całe drzewo, od korzenia aż do liści
    for val in possible_values(df, best_column):
        filtered_df = df[df[best_column] == val]
        child_id = f"{node_id}_{val}"
        build_tree(filtered_df, tree, remaining_columns, node_id, child_id)


# Ta sama funkcja, ale buduje drzewo razem z liśćmi zawierającymi decyzję
def build_tree_with_decision(df, tree, columns, parent=None, parent_id="root"):

    # Jeśli decyzja jest 100% pewna, to liść
    if len(df['Survived'].unique()) == 1:
        decision = df['Survived'].values[0]
        tree.create_node(f"{decision}", parent_id, parent=parent)
        return

    # reszta funkcji jest dokładnie taka sama
    best_gain = 0.0
    best_column = None
    for column in columns:
        ig = information_gain(split(df, column))
        if ig > best_gain:
            best_gain = ig
            best_column = column
    if best_gain == 0 or len(columns) == 0:
        return
    remaining_columns = columns.copy()
    remaining_columns.remove(best_column)
    node_id = f"{best_column}_{parent_id}"
    tree.create_node(best_column, node_id, parent=parent)
    for val in possible_values(df, best_column):
        filtered_df = df[df[best_column] == val]
        child_id = f"{node_id}_{val}"
        build_tree_with_decision(filtered_df, tree, remaining_columns, node_id, child_id)


Ostatecznie, driver code, czyli wywołanie przygotowanych powyżej funkcji:

In [26]:
from treelib import Tree

dane = prepare_pandas_df_for_basic_problem("titanic-homework.csv")
decision_tree = Tree()
columns = get_column_names(dane)
build_tree_with_decision(dane, decision_tree, columns)
print(decision_tree.show(stdout=False))

print("======Separator======\n")

decision_tree2 = Tree()
build_tree(dane, decision_tree2, columns)
print(decision_tree2.show(stdout=False))

Sex
├── Pclass
│   ├── Age
│   │   ├── 0
│   │   ├── SibSp
│   │   │   ├── 0
│   │   │   └── 1
│   │   └── SibSp
│   │       ├── 0
│   │       └── 1
│   ├── Age
│   │   ├── 0
│   │   ├── 0
│   │   └── 1
│   └── Parch
│       ├── 0
│       ├── 0
│       ├── 0
│       ├── 0
│       └── Age
│           ├── 0
│           └── 1
└── SibSp
    ├── 0
    ├── 0
    ├── 1
    ├── 1
    ├── Age
    │   ├── 0
    │   └── 1
    └── Pclass
        ├── 1
        ├── 1
        └── Parch
            ├── 0
            ├── 1
            └── 1

======Separator======

Sex
├── Pclass
│   ├── Age
│   │   ├── SibSp
│   │   └── SibSp
│   ├── Age
│   └── Parch
│       └── Age
└── SibSp
    ├── Age
    └── Pclass
        └── Parch



# Wersja Pro

Większość funkcji pozostaje taka sama, jednak potrzebna jest dodatkowa funkcja, która wyznaczy granice decyzyjne na atrybucie "Age"

In [27]:
# todo